In [1]:
import pandas as pd
payment_df = pd.read_csv("Payment_db_data_npia.csv")
payment_df.head()

C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,payment_id,company_id,type,mode,amount,date,status,reference,note,payment_context,...,customer_id,vendor_id,bank_account_id,bank_upi_id,estimate_id,createdBy,updatedBy,created_at,updated_at,customer_name
0,1,1,OUT,OTHER,55507,01-04-2023 00:00,MANUAL,Invoice,Digimatic dial gauge with accessories,PURCHASE,...,NaN,NaN,NaN,NaN,NaN,1,1,55:47.7,55:47.7,Shree shyam traders
1,2,1,OUT,OTHER,68800,07-04-2023 00:00,MANUAL,No attachment,Rent for april 2023 (ATS-RH07),PURCHASE,...,NaN,NaN,NaN,NaN,NaN,1,1,55:47.7,55:47.7,Rainder kaur walia
2,3,1,OUT,OTHER,10080,17-04-2023 00:00,MANUAL,Invoice copy,Reimbursment for travel exp (Radisson Hotel),PURCHASE,...,NaN,NaN,NaN,NaN,NaN,1,1,55:47.7,55:47.7,Gnaneshwar Shubhakumar
3,4,1,OUT,OTHER,3168,17-04-2023 00:00,MANUAL,Invoice,"Payment against wifi adaptor, ups",PURCHASE,...,NaN,NaN,NaN,NaN,NaN,1,1,55:47.7,55:47.7,SRJ Technologies
4,5,1,OUT,OTHER,444000,18-04-2023 00:00,MANUAL,Proforma,30% advance payment for work scope as per BOQ,PURCHASE,...,NaN,NaN,NaN,NaN,NaN,1,1,55:47.7,55:47.7,Atharv industries corporation


In [4]:
bank_statement_df = pd.read_csv("NPI_feature_extraction.csv")
bank_statement_df.head()

,txn_date,value_date,description,remark_(npia),ref_no./cheque_no.,branch_code,debit,credit,balance,sender_or_reciever_info,transaction_type,currency,exchange_rate
0,03-04-2023,03-04-2023,TO TRANSFER-INB NEFT UTR NO: SBIN2230936145...,RENT ATS,NEFT INB: CNACKGAGQ9 ...,99922,70100,,51322448.45,NIMISHA AHUJA,NEFT,NaN,NaN
1,03-04-2023,03-04-2023,TO TRANSFER-INTEREST PAID--,Bank Interest,TRANSFER TO 41541829866 ...,3024,3365000,,47957448.45,NaN,NaN,NaN,NaN
2,04-04-2023,04-04-2023,TO TRANSFER-INB NEFT UTR NO: SBIN4230940432...,Food Materials,NEFT INB: CNACKISXZ2 ...,99922,55507,,47901941.45,Shree shyam traders,NEFT,NaN,NaN
3,05-04-2023,05-04-2023,BY TRANSFER-INB IMPS309522961613/8800772299...,configuration,MAD000278742610 MAD000278742610 ...,99922,,100,47902041.45,8800772299,IMPS,NaN,NaN
4,07-04-2023,07-04-2023,TO TRANSFER-INB RTGS UTR NO: SBINR120230407...,Stamp paper for Company Contract with Jahwa,RTGS INB: CRE3732647 ...,99922,240000,,47662041.45,SHCIL ESTAMPING,RTGS,NaN,NaN


In [6]:
print(payment_df.columns)
print(bank_statement_df.columns)

Index(['payment_id', 'company_id', 'type', 'mode', 'amount', 'date', 'status',
       'reference', 'note', 'payment_context', 'receipt_url', 'customer_id',
       'vendor_id', 'bank_account_id', 'bank_upi_id', 'estimate_id',
       'createdBy', 'updatedBy', 'created_at', 'updated_at', 'customer_name'],
      dtype='object')
Index(['txn_date', 'value_date', 'description', 'remark_(npia)',
       'ref_no./cheque_no.', 'branch_code', 'debit', 'credit', 'balance',
       'sender_or_reciever_info', 'transaction_type', 'currency',
       'exchange_rate'],
      dtype='object')


In [28]:
import pandas as pd
from rapidfuzz import fuzz
from datetime import timedelta

# Ensure dates are in datetime format
payment_df['date'] = pd.to_datetime(payment_df['date'], dayfirst=True, errors='coerce')
bank_statement_df['txn_date'] = pd.to_datetime(bank_statement_df['txn_date'], dayfirst=True, errors='coerce')

# Fill missing values and lowercase for comparison
payment_df['customer_name'] = payment_df['customer_name'].fillna('').str.lower()
bank_statement_df['sender_or_reciever_info'] = bank_statement_df['sender_or_reciever_info'].fillna('').str.lower()

# Remove commas (if any), and convert debit/credit to numeric
bank_statement_df['debit'] = pd.to_numeric(bank_statement_df['debit'].astype(str).str.replace(',', ''), errors='coerce')
bank_statement_df['credit'] = pd.to_numeric(bank_statement_df['credit'].astype(str).str.replace(',', ''), errors='coerce')

# Also make sure payment amount is numeric
payment_df['amount'] = pd.to_numeric(payment_df['amount'], errors='coerce')

# Parameters
amount_tolerance = 5               # +/- 5 currency units
date_tolerance_days = 30           # +/- 2 days
name_similarity_threshold = 60     # percent match for fuzzy string comparison

matches = []

# Loop through payments
for _, payment in payment_df.iterrows():
    for _, bank_txn in bank_statement_df.iterrows():
        txn_amount = bank_txn['credit'] if payment['type'] == 'IN' else bank_txn['debit']
        
        if pd.isna(txn_amount):
            continue
        
        # Amount check
        if abs(payment['amount'] - txn_amount) > amount_tolerance:
            continue
        
        # Date check
        if abs((payment['date'] - bank_txn['txn_date']).days) > date_tolerance_days:
            continue
        
        # Fuzzy name matching
        similarity = fuzz.token_sort_ratio(payment['customer_name'], bank_txn['sender_or_reciever_info'])
        if similarity < name_similarity_threshold:
            continue

        # Save match
        matches.append({
            'payment_id': payment['payment_id'],
            'payment_date': payment['date'],
            'payment_amount': payment['amount'],
            'customer_name': payment['customer_name'],
            'bank_txn_date': bank_txn['txn_date'],
            'statement_reference_no':bank_txn['ref_no./cheque_no.'],
            'bank_amount': txn_amount,
            'sender_or_reciever_info': bank_txn['sender_or_reciever_info'],
            'similarity': similarity
        })

# Create DataFrame with matches
matched_df = pd.DataFrame(matches).sort_values(by='similarity', ascending=False)


In [30]:
matched_df

,payment_id,payment_date,payment_amount,customer_name,bank_txn_date,statement_reference_no,bank_amount,sender_or_reciever_info,similarity
0,1,2023-04-01,55507,shree shyam traders,2023-04-04,NEFT INB: CNACKISXZ2 ...,55507.00,shree shyam traders,100.000000
12,14,2023-04-24,11210,calcify it solutions,2023-04-24,NEFT INB: CNACLBXUO4 ...,11210.00,calcify it solutions,100.000000
2,4,2023-04-17,3168,srj technologies,2023-04-18,NEFT INB: CNACKWKTN1 ...,3168.00,srj technologies,100.000000
3,5,2023-04-18,444000,atharv industries corporation,2023-04-18,RTGS INB: CRE4098188 ...,444000.00,atharv industries corporation,100.000000
16,19,2023-04-26,48498,booyoung india pvt ltd,2023-04-28,NEFT INB: CNACLGCPA5 ...,48498.00,booyoung india pvt ltd,100.000000
14,17,2023-04-25,125473,techpc7,2023-04-25,NEFT INB: CNACLDRKO3 ...,125473.00,techpc7,100.000000
6,8,2023-04-19,5237,gps office equipment pvt ltd,2023-04-20,NEFT INB: CNACKYPNF9 ...,5237.40,gps office equipment pvt ltd,100.000000
7,9,2023-04-19,2832,greno enterprises,2023-04-20,NEFT INB: CNACKYRPG7 ...,2832.00,greno enterprises,100.000000
8,10,2023-04-20,64055,a2a green technologies llp,2023-04-20,NEFT INB: CNACKZKIO3 ...,64054.64,a2a green technologies llp,100.000000
1,2,2023-04-07,68800,rainder kaur walia,2023-04-07,NEFT INB: CNACKMZIA0 ...,68800.00,rainder kaur walia,100.000000


In [14]:
bank_statement_df

,txn_date,value_date,description,remark_(npia),ref_no./cheque_no.,branch_code,debit,credit,balance,sender_or_reciever_info,transaction_type,currency,exchange_rate
0,2023-04-03,03-04-2023,TO TRANSFER-INB NEFT UTR NO: SBIN2230936145...,RENT ATS,NEFT INB: CNACKGAGQ9 ...,99922,70100.00,NaN,51322448.45,nimisha ahuja,NEFT,NaN,NaN
1,2023-04-03,03-04-2023,TO TRANSFER-INTEREST PAID--,Bank Interest,TRANSFER TO 41541829866 ...,3024,3365000.00,NaN,47957448.45,,NaN,NaN,NaN
2,2023-04-04,04-04-2023,TO TRANSFER-INB NEFT UTR NO: SBIN4230940432...,Food Materials,NEFT INB: CNACKISXZ2 ...,99922,55507.00,NaN,47901941.45,shree shyam traders,NEFT,NaN,NaN
3,2023-04-05,05-04-2023,BY TRANSFER-INB IMPS309522961613/8800772299...,configuration,MAD000278742610 MAD000278742610 ...,99922,NaN,100.0,47902041.45,8800772299,IMPS,NaN,NaN
4,2023-04-07,07-04-2023,TO TRANSFER-INB RTGS UTR NO: SBINR120230407...,Stamp paper for Company Contract with Jahwa,RTGS INB: CRE3732647 ...,99922,240000.00,NaN,47662041.45,shcil estamping,RTGS,NaN,NaN
5,2023-04-07,07-04-2023,TO TRANSFER-INB NEFT UTR NO: SBIN3230973730...,RH07 Rent,NEFT INB: CNACKMZIA0 ...,99922,68800.00,NaN,47593241.45,rainder kaur walia,NEFT,NaN,NaN
6,2023-04-08,08-04-2023,TO TRANSFER-INB RTGS UTR NO: SBINR120230408...,Salary,RTGS INB: CRE3751603 ...,99922,250000.00,NaN,47343241.45,jong wook noh,RTGS,NaN,NaN
7,2023-04-10,10-04-2023,BY TRANSFER-INB IMPS310017952183/8800772299...,Security Return,MAB001172452318 MAB001172452318 ...,99922,NaN,13000.0,47356241.45,8800772299,IMPS,NaN,NaN
8,2023-04-10,10-04-2023,BY TRANSFER-INB IMPS310017725993/8800772299...,Security Return,MAD000281789707 MAD000281789707 ...,99922,NaN,13000.0,47369241.45,8800772299,IMPS,NaN,NaN
9,2023-04-12,12-04-2023,BY TRANSFER-RTGS UTR NO: SHBKR5202304120494...,Shareholder Investment,TRANSFER FROM 3199856044300 / JINWOO KIM,4430,NaN,1000000.0,48369241.45,jinwoo kim,RTGS,NaN,NaN


In [10]:
payment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   payment_id       23 non-null     int64         
 1   company_id       23 non-null     int64         
 2   type             23 non-null     object        
 3   mode             23 non-null     object        
 4   amount           23 non-null     int64         
 5   date             23 non-null     datetime64[ns]
 6   status           23 non-null     object        
 7   reference        22 non-null     object        
 8   note             23 non-null     object        
 9   payment_context  23 non-null     object        
 10  receipt_url      0 non-null      float64       
 11  customer_id      0 non-null      float64       
 12  vendor_id        0 non-null      float64       
 13  bank_account_id  0 non-null      float64       
 14  bank_upi_id      0 non-null      float64    

In [17]:
matched_df.payment_id.to_list()

[2, 10, 19, 17, 15, 14, 4, 12, 9, 8, 5, 11, 20, 18, 6, 13, 21, 7]

In [29]:
result = payment_df[~payment_df['payment_id'].isin(matched_df['payment_id'])]
result

,payment_id,company_id,type,mode,amount,date,status,reference,note,payment_context,...,customer_id,vendor_id,bank_account_id,bank_upi_id,estimate_id,createdBy,updatedBy,created_at,updated_at,customer_name
2,3,1,OUT,OTHER,10080,2023-04-17,MANUAL,Invoice copy,Reimbursment for travel exp (Radisson Hotel),PURCHASE,...,NaN,NaN,NaN,NaN,NaN,1,1,55:47.7,55:47.7,gnaneshwar shubhakumar
15,16,1,OUT,OTHER,1690924,2023-04-24,MANUAL,Proforma,Advance payment against structural vetting by IIT,PURCHASE,...,NaN,NaN,NaN,NaN,NaN,1,1,55:47.7,55:47.7,foundation to structure (f2s)
21,22,1,OUT,OTHER,585000,2023-05-02,MANUAL,Invoice,Office interior work,PURCHASE,...,NaN,NaN,NaN,NaN,NaN,1,1,55:47.7,55:47.7,atharv industries corporation
22,23,1,OUT,OTHER,202351,2023-05-03,MANUAL,NaN,"Reimbursement to CFO (office goods, foods,etc)...",PURCHASE,...,NaN,NaN,NaN,NaN,NaN,1,1,55:47.7,55:47.7,hyeong seok jeong


In [ ]:
# match bases on amount, customer name and then date.
# calculate confidence score based on the data above and show recommendation to user
# from payment table show use all avaibale unmatched bank statement records if confidence score is very low

# make prompt for all cases to match invoice with payment table. return output as csv with 100% matched at top, then partially matched data below them

In [3]:
#!pip install difflib

import pandas as pd
payment_df = pd.read_csv("Payment_db_data_npia.csv")
payment_df.head()

bank_statement_df = pd.read_csv("NPI_feature_extraction.csv")
bank_statement_df.head()


,txn_date,value_date,description,remark_(npia),ref_no./cheque_no.,branch_code,debit,credit,balance,sender_or_reciever_info,transaction_type,currency,exchange_rate
0,03-04-2023,03-04-2023,TO TRANSFER-INB NEFT UTR NO: SBIN2230936145...,RENT ATS,NEFT INB: CNACKGAGQ9 ...,99922,70100,,51322448.45,NIMISHA AHUJA,NEFT,NaN,NaN
1,03-04-2023,03-04-2023,TO TRANSFER-INTEREST PAID--,Bank Interest,TRANSFER TO 41541829866 ...,3024,3365000,,47957448.45,NaN,NaN,NaN,NaN
2,04-04-2023,04-04-2023,TO TRANSFER-INB NEFT UTR NO: SBIN4230940432...,Food Materials,NEFT INB: CNACKISXZ2 ...,99922,55507,,47901941.45,Shree shyam traders,NEFT,NaN,NaN
3,05-04-2023,05-04-2023,BY TRANSFER-INB IMPS309522961613/8800772299...,configuration,MAD000278742610 MAD000278742610 ...,99922,,100,47902041.45,8800772299,IMPS,NaN,NaN
4,07-04-2023,07-04-2023,TO TRANSFER-INB RTGS UTR NO: SBINR120230407...,Stamp paper for Company Contract with Jahwa,RTGS INB: CRE3732647 ...,99922,240000,,47662041.45,SHCIL ESTAMPING,RTGS,NaN,NaN


In [5]:
import pandas as pd
from difflib import SequenceMatcher

# --- Preprocessing ---
payment_df['date'] = pd.to_datetime(payment_df['date'], dayfirst=True, errors='coerce')
bank_statement_df['txn_date'] = pd.to_datetime(bank_statement_df['txn_date'], dayfirst=True, errors='coerce')

payment_df['customer_name'] = payment_df['customer_name'].fillna('').str.lower().str.strip()
bank_statement_df['sender_or_reciever_info'] = bank_statement_df['sender_or_reciever_info'].fillna('').str.lower().str.strip()

bank_statement_df['debit'] = pd.to_numeric(
    bank_statement_df['debit'].astype(str).str.replace(',', ''), errors='coerce')
bank_statement_df['credit'] = pd.to_numeric(
    bank_statement_df['credit'].astype(str).str.replace(',', ''), errors='coerce')
payment_df['amount'] = pd.to_numeric(payment_df['amount'], errors='coerce')

bank_statement_df['description'] = bank_statement_df['description'].fillna('').str.lower()
payment_df['reference'] = payment_df['reference'].fillna('').str.lower()

# --- Name Similarity ---
def get_name_match_score(name1, name2):
    if not name1 or not name2:
        return 0

    name1_clean = name1.replace('(', '').replace(')', '')
    name2_clean = name2.replace('(', '').replace(')', '')

    # Token containment logic
    if name2_clean in name1_clean or name1_clean in name2_clean:
        return 0.6  # Strong match if one is substring of the other

    # Fallback: original logic
    name1_tokens = set(name1_clean.split())
    name2_tokens = set(name2_clean.split())
    token_overlap = len(name1_tokens & name2_tokens) / max(len(name1_tokens | name2_tokens), 1)
    string_similarity = SequenceMatcher(None, name1_clean, name2_clean).ratio()
    return max(token_overlap, string_similarity)


# --- Match Score ---
def calculate_match_score(payment, bank_txn):
    if payment['date'] > bank_txn['txn_date']:
        return 0, None, None, 0, False

    amount_diff = abs((bank_txn['debit'] or bank_txn['credit'] or 0) - payment['amount']) if pd.notna(bank_txn['debit']) else float('inf')
    date_diff = (bank_txn['txn_date'] - payment['date']).days
    name_score = get_name_match_score(payment['customer_name'], bank_txn['sender_or_reciever_info'])
    ref_match = payment['reference'] in bank_txn['description']

    # High confidence if name and amount are strong matches
    if name_score >= 0.85 and amount_diff <= 1:
        base_score = 0.95
    elif name_score >= 0.7 and amount_diff == 0:
        base_score = 0.9
    elif name_score >= 0.5 and amount_diff == 0:
        base_score = 0.7
    elif name_score >= 0.6 and amount_diff <= 3:
        base_score = 0.6
    else:
        return 0, amount_diff, date_diff, name_score, ref_match  # Not confident enough

    # Add minor boosts
    if date_diff <= 5:
        base_score += 0.03
    if ref_match:
        base_score += 0.02

    return round(min(base_score, 1.0), 4), amount_diff, date_diff, round(name_score, 4), ref_match

# --- Match Finder: Best per Payment ---
matches = []

for _, payment in payment_df.iterrows():
    best_score = 0
    best_match = None

    for _, bank_txn in bank_statement_df.iterrows():
        if bank_txn['txn_date'] < payment['date']:
            continue  # only match txn that occurs on or after payment

        score, amt_diff, dt_diff, name_sim, ref_match = calculate_match_score(payment, bank_txn)

        # Choose best score, and break ties by earlier bank_txn_date
        if (score > best_score) or (score == best_score and best_match and bank_txn['txn_date'] < best_match['bank_txn_date']):
            best_score = score
            best_match = {
                'payment_id': payment['payment_id'],
                'bank_ref_no': bank_txn['ref_no./cheque_no.'],
                'match_score': score,
                'amount_diff': amt_diff,
                'date_diff_days': dt_diff,
                'name_match_score': name_sim,
                'reference_match': ref_match,
                'bank_txn_date': bank_txn['txn_date'],
                'payment_date': payment['date'],
                'payment_amount': payment['amount'],
                'bank_debit': bank_txn['debit'],
                'customer_name': payment['customer_name'],
                'bank_party': bank_txn['sender_or_reciever_info']
            }

    if best_match and best_score > 0.6:
        matches.append(best_match)
        


# --- Export to CSV ---
matched_df = pd.DataFrame(matches).sort_values(by='match_score', ascending=False).reset_index(drop=True)
matched_df.to_csv("best_matched_payments.csv", index=False)
print("Saved high-confidence matches to 'best_matched_payments.csv'")


Saved high-confidence matches to 'best_matched_payments.csv'


In [6]:
result = payment_df[~payment_df['payment_id'].isin(matched_df['payment_id'])]
result

,payment_id,company_id,type,mode,amount,date,status,reference,note,payment_context,...,customer_id,vendor_id,bank_account_id,bank_upi_id,estimate_id,createdBy,updatedBy,created_at,updated_at,customer_name
17,18,1,OUT,OTHER,10974,2023-04-26,MANUAL,invoice,Material for production department,PURCHASE,...,NaN,NaN,NaN,NaN,NaN,1,1,55:47.7,55:47.7,ins-tech engineer's
21,22,1,OUT,OTHER,585000,2023-05-02,MANUAL,invoice,Office interior work,PURCHASE,...,NaN,NaN,NaN,NaN,NaN,1,1,55:47.7,55:47.7,atharv industries corporation
22,23,1,OUT,OTHER,202351,2023-05-03,MANUAL,,"Reimbursement to CFO (office goods, foods,etc)...",PURCHASE,...,NaN,NaN,NaN,NaN,NaN,1,1,55:47.7,55:47.7,hyeong seok jeong
